In [11]:
import yfinance as yf
import pandas as pd
import numpy
import math
import altair as alt
import time
import json
import datetime
import sqlite3
import multiprocessing  
import random
from multiprocessing import Pool
import defs
import time
import datetime
import requests
import aiohttp
import asyncio

import re

import requests_cache
#requests_cache.install_cache(cache_name="cache_cache", backend='sqlite')
requests_cache.uninstall_cache()

def Logging(val):
    with open("log.txt") as file: # Use file to refer to the file object
        file.write(val)


#missing777
#fees
#dividends
#stocksplits - what happens in  a stock split(are old shares amount converted to new share amount)
#confirm values, see if value is between open and close of that day
#drop columns when amount is 0

def return_print(msg="",payload=None,status="INFO"):
    
    payload={}
    if payload is None:
        payload=payload

    payload['status']=status
    payload['msg']=msg
    
    print(json.dumps(payload))

from random_proxies import random_proxy


#TODO
#fees 
#dividends

You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [16]:
#cross results from summaries with results from download
filename = 'Finance_Positions.csv'

f = open(filename, "r")

count_comma=0
count_semicolon=0
for i in range(2):
    line = f.readline()
    count_comma=count_comma+line.count(",")
    count_semicolon=count_semicolon+line.count(";")

f.close()

print("Importing Transactions")

sep=","
if(count_semicolon>count_comma):
    sep=";"

#transactions list
transactions = pd.read_csv(filename,sep=sep)

#get the most earliest date
df=pd.DataFrame()
df['Date']=transactions['When']

#the start date
startdate=pd.to_datetime(df['Date']).sort_values()[0].strftime("%Y-%m-%d")

#get all use tickers distinct
stonks=pd.unique(transactions['Ticker']).tolist()

print("Transactions Loaded")

Importing Transactions
Transactions Loaded


In [32]:
#check if in db

con = sqlite3.connect("example.db")
cursy = con.cursor()
    
stonk_summaries={}
stonk_summaries_missing =[]

#consider paraleliszing
for s in stonks:
    
    print(s)
    
    cursy.execute('SELECT * FROM stocks_summary where ticker==?',(s,))
    #df = pd.read_sql_query("SELECT * from stocks_summary where ticker = ?".formar(, con)

    rows = cursy.fetchone()
    if rows is not None:
        stonk_summaries[rows[0]]=json.loads(rows[1])
    else:
        stonk_summaries_missing.append(s)
    
    print(rows)
cursy.close()
con.close()

2
None
SEED
None


In [33]:
stonk_summaries
stonk_summaries_missing

['2', 'SEED']

In [36]:
#get from requests
import requests

lst = ['AAPL']
response = requests.post('http://localhost:3000/api/stoccies', json={'stonks':stonk_summaries_missing})
response.content

#print errors:
print("ERRORES ARE:",json.loads(response.content)['ERRORS'])

['2']


In [ ]:
import json
summaries={}
with open('merger.json') as f:
  summaries = json.load(f)


In [ ]:
json.dumps(summaries['XOM']['summaryDetail']['currency'])

In [ ]:
data = yf.download(tickers=['TSLA'],start='2020-08-05',end='2020-09-05',interval='1d',group_by='column',prepos=False,threads=True)

In [ ]:


def SaveSummaryToDB(summaries):
    con = sqlite3.connect("example.db")
    cursy = con.cursor()

    for key in summaries:
        print(key)

        try:               

            #cursy.execute('INSERT INTO stocks_summary(ticker,summary) VALUES (?,?)', ( key, json.dumps(summaries[key])))
            cursy.execute('INSERT INTO stocks_currency(ticker,currency) VALUES (?,?)', ( key,summaries[key]['summaryDetail']['currency']))

        except Exception as e:
            print(e)
            cursy.execute('INSERT INTO error_log VALUES (?,?)', ( str(e),))

            print("Whoopsie")

    con.commit()
    cursy.close()
    con.close()

In [ ]:
#call  summary api 

#check db, if not, call api

import requests


response = requests.post('http://localhost:3000/api/stoccies', data={'stonks':['TSLA']})

In [ ]:
response.content

In [ ]:
# Read sqlite query results into a pandas DataFrame
#load missing
con = sqlite3.connect("example.db")
df = pd.read_sql_query("SELECT * from stocks_currency", con)

c = set(stonks).difference(set(df.ticker))
df = pd.read_sql_query("SELECT * from stocks_error", con)
#c = set(c).difference(set(df.ticker))
stonks_missing=list(c)
len(stonks_missing)

In [ ]:

def log_result(result):
    # This is called whenever foo_pool(i) returns a result.
    # result_list is modified only by the main process, not the pool workers.

    try:
        #inf=defs.FetchTickerInfo(i)

        print("Fetched")

        cony = sqlite3.connect('example.db')
        cursy = cony.cursor()
        print("Inserting")
        print(result[0])

        x = re.findall("\"currency\":\"(.*?)\"", result[1].decode('UTF-8'))#.decode('UTF-8')
        
        if len(x)>0:
            cursy.execute('INSERT INTO stocks_currency VALUES (?,?)', ( result[0], x[0]))
            cony.commit()
            print(result[0],x)
        else:
            cursy.execute('INSERT INTO stocks_error VALUES (?)', (result[0],))
            cony.commit()
            print(result[0],"err")
    except Exception as e:
        print(e)
        print("Whoopsie")



#multithread load of stocks not already obtained 
infos=[]
if __name__ == '__main__':
    pool = Pool(32)
    #infos = pool.map(defs.FetchTickerInfo, stonks_missing)

    count=0
    for i in stonks_missing:
        count=count+1
        print(count)
        pool.apply_async(defs.FetchTickerInfo3, args = (i, ), callback = log_result)
    print("Thread Started")
    pool.close()
    pool.join()
    #print(result_list)

#get current format
curtime=datetime.datetime.now().isoformat()

stonks_to_add = []

for tick, inf in zip(stonks_missing,infos):


    stonks_info[tick] = inf
    stonks_to_add.append((tick, json.dumps(inf),curtime))

#save to avoid future big times
cur.executemany('INSERT INTO stocks_info VALUES (?,?,?)', stonks_to_add)


#T16 - 13
#T4 - 26 
print(time.time()-start)

con.commit()
con.close()


In [ ]:
res = defs.FetchTickerInfo3("TPOP.L")



In [ ]:
res[1]

In [ ]:
x = re.findall("\"currency\":\"(.*?)\"", res[1].decode('UTF-8'))#.

In [ ]:
x

In [ ]:

#donwload all data
data = yf.download(tickers=stonks,start=startdate,interval='1d',group_by='column',prepos=False,threads=True)

pairs = yf.download(tickers="EURUSD=X",start=startdate,interval='1d',group_by='column',prepos=False,threads=True)

#counts entries without values
empty_stocks  =data['Close'].columns[data['Close'].count()==0].to_list()

if len(empty_stocks)>0:
    payload= {}
    payload['stocks'] = empty_stocks
    return_print("The Following Tickers are Invalid:"+" ".join(empty_stocks)+"\n, tickers must be set accordingly to  https://finance.yahoo.com/",payload,status='ERROR')

return_print("Stock Values Loaded")


In [ ]:
stonkdict={}


targetcurrency='EUR'

superdf =pd.DataFrame()
print(superdf.empty)

i=0


for stonk in stonks:  
    
    print(stonk)
    
    #get transactions from csv
    singlestock_transactions = transactions[transactions['Ticker']==stonk]

    #use index to create df
    index = data.index
    df=pd.DataFrame(index=data.index)

    
    #extract price
    df['Price']= data['Close'][stonk]
    #initialize amount
    df["SharesOwned"]=0
    #initialize total value
    df['TotalValue']=0   

    df['InvestedValue']=0   

    #set ticker
    df["Ticker"]=stonk
    
    #fetch currency
    df["Currency"]=json.loads(stonks_info['DIOD'][1])['currency']


    for idx, row in singlestock_transactions.iterrows():

        multiplier = 1
        if(row['What']=='buy'):
            multiplier=1
        elif(row['What']=='sell'):
            multiplier=-1
        else:
            raise Exception("Transaction \"row['What']\" is not recognized")

        #print(row['Amount'])


        #CHECK IF IT GETS APPLIED TO STARTING ARRAY, FIND AWAY TO DO THIS AT THE START
        #CONVERTING INTO AMOUNT MAKES it be adimensional, and doenst matter what currency it was bought in
        try:
            if(math.isnan(float(row['Amount']))):
                row['Amount']=row['Total Cost']/row['Cost per Share']
        except:
            row['Amount']=row['Total Cost']/row['Cost per Share']


        #df[row['When']:][stonk+" Amount"] = df[row['When']:][stonk+" Amount"]=row['Amount']

        #df[row['When']:,stonk+" Amount"] = df[row['When']:,stonk+" Amount"]+row['Amount']

        #print(float(row['Amount']))
        
        if pd.to_datetime("now")<pd.to_datetime(row['When']):
            print(row['When'],"<- This date is in the future, and will not be considere")
            continue


        
        #print(float(row['Amount'])*multiplier)
        #set amount to the amount
        #display(row['When'])
        #display(df.loc[row['When']:])
        
        
        
        
        #change the amount from whtn to forward,  multiplier determinex if its a  buy of a sell
        df.loc[row['When']:,'SharesOwned'] = df.loc[row['When']:,'SharesOwned'] + (float(row['Amount'])*multiplier)

        #money put in
        df.loc[row['When']:,'InvestedValue'] = df.loc[row['When']:,'InvestedValue'] + (float(row['Total Cost'])*multiplier)
        
        
        
        

        

    i=i+1
        
    df.loc[:,'TotalValue']= df["Price"]*df['SharesOwned']
    df.loc[:,'EarnedValue']= df.loc[:,'TotalValue']-df.loc[:,'InvestedValue']
    
    
        
    
    
    
    
    #delete rows with no data
    #
    df.interpolate(inplace=True)
    df.dropna(inplace=True,how='any')
    
    
    #display(df)
    #display(df)
    
    superdf= superdf.append(df)
    #display(df)
    #display(superdf)
    #print("YIKES")

    




In [ ]:
superdf['CurrencyUSD']=0

In [ ]:
fauxpair = pairs.loc[:]['Close']

In [ ]:


hm = pd.merge(
    superdf,
    fauxpair,
    how="left",
    left_on='Date',
    right_on='Date',
    left_index=False,
    right_index=False,
    suffixes=("_x", "_y"),
    copy=True,
    indicator=False,
    validate=None,
)

hm.loc[:]['Close']=hm.loc[:]['Close'].interpolate()




In [ ]:
import sqlite3
con = sqlite3.connect('example.db')

cur = con.cursor()

# Create table
cur.execute('''CREATE TABLE stocks_info (ticker text, info text,dateinserted TEXT )''')

               # Save (commit) the changes
#cur.execute('DROP TABLE stocks_info')
#cur.execute('DROP TABLE stocks')

con.commit()

# We can also close the connection if we are done with it.
# Just be sure any changes have been committed or they will be lost.
con.close()

In [ ]:
from tinydb import TinyDB, Query
db = TinyDB('db.json')
db.insert({'type': 'apple', 'count': 7})

In [ ]:
!pip install tinydb

#df.loc[row['When']:,'Amount'] = df.loc[row['When']:,'Amount'] + (float(row['Amount'])*multiplier)
hm.loc[:,hm['Currency']=='EUR'] = hm.loc[:,'Close']*hm.loc[:,'TotalValue']

In [ ]:
#selecct column based on another column
hm.loc[hm['Currency']=='EUR',['Currency']]


#hm.loc[:,hm['Currency']=='EUR']['Close']
#hm[hm['Currency']=='EUR']['Currency']
#rectify us currency

#convert currency
hm.loc[:,'TotalValueUSD']=0
hm.loc[:,'TotalValueUSD']=hm.loc[:,'TotalValue']
hm.loc[hm['Currency']=='EUR','TotalValueUSD'] = hm[hm['Currency']=='EUR']['TotalValue']*hm[hm['Currency']=='EUR']['Close']

In [ ]:
print(time.time()-startu)

In [ ]:
#superdf[superdf['Ticker']=='XOM']
dfforaltair = superdf.loc["2019-10-20":].reset_index()
dfforaltair.interpolate(inplace=True)

alt.data_transformers.disable_max_rows()
alt.Chart(dfforaltair).mark_area().encode(
    x='Date',
    y='TotalValue',
    #color='Ticker:N',
    color=alt.Color('Ticker:N', legend=alt.Legend(columns=1,symbolLimit=300)),
).properties(
    width=1280,
    height=720
)

In [ ]:
for key in stonkdict:
    print(key)

702*0.50


df.loc['04/02/2020':, 'TSLAR'] = 2

df

In [ ]:
#df[df['1962-01-02':]][stonk+" Amount"]=df[df['1962-01-02':]][stonk+" Amount"]+row['Amount']



df

In [ ]:
!pip install --upgrade numpy
!pip install stocker